In [10]:
import sys 
sys.path.append('../..')

import math
import numpy as np
import torch
import torch

import gpytorch
from gpytorch.means import ZeroMean
from gpytorch.kernels import RBFKernel, PeriodicKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import VariationalELBO, PredictiveLogLikelihood

from horseshoe_gp.src.structural_sgp import VariationalGP, StructuralSparseGP, \
TrivialSelector, SpikeAndSlabSelector, SpikeAndSlabSelectorV2, HorseshoeSelector
from horseshoe_gp.src.mean_field_hs import MeanFieldHorseshoe, VariatioalHorseshoe

import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from scipy.optimize import minimize, Bounds

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(
    boston['data'],
    boston['target'],
    test_size=0.2,
    random_state=2020)

#I am not sure scaling per train and test is appropriate
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)
y_train = StandardScaler().fit_transform(np.expand_dims(y_train, axis=1))
y_test = StandardScaler().fit_transform(np.expand_dims(y_test, axis=1))

#set up kernels
n_kernels = 5
means = [ZeroMean()] * n_kernels
kernels = [RBFKernel()] * n_kernels

n_inducing = 50
inducing_points = torch.linspace(0, 1, n_inducing)

# GP for each kernel
gps = []
for mean, kernel in zip(means, kernels):
    gp = VariationalGP(mean, kernel, inducing_points)
    gps.append(gp)
    
norm = torch.distributions.normal.Normal(0, 1)

SVM regression on Boston housing dataset

In [11]:
def svm_reg_evaluate(param_log_eps, param_log_gamma, param_C):
    model = svm.SVR(epsilon=param_log_eps.exp(),
                   gamma=param_log_gamma.exp(),
                   C=param_C,
                   kernel='rbf')
    model.fit(X_train, Y_train)
    return mean_squared_error(model.predict(X_test, Y_test), squared = True)

num_param = 3
num_start_pnt = 5

#log_eps, log_gamma, C
lb = np.array([0, 0, 0])
ub = np.array([-5, -5, 10000])

scope = Bounds(lb, ub)

In [15]:
def UCB(x, sur_model, kappa = 2.576):
    mean, std = sur_model(x)
    return mean + kappa * std

def EI(x, sur_model, y_max):
    mean, std = sur_model(x)
    a = (mean - ymax - x)
    z = a / std
    return a * norm.cdf(z) + std * norm.pdf(z)

def POI(x, sur_model, y_max):
    mean, std = sur_model(x)
    z = (mean - y_max - x)/std
    return norm.cdf(z)

def acq_max(bounds, sur_model, y_max, acq_fun, n_warmup = 10000, iteration = 10):
    x_tries = torch.empty(n_warmup, bounds.shape[0])._rand() * (bounds[:, 1] - bounds[:, 0]) + bounds[:, 0]
    ys = acq_fun(x_tries, sur_model, y_max, kappa)
    x_max = x_tries[ys.argmax()]
    max_acq = ys.max()
    
    for iterate in range(iteration):
        locs = torch.empty(bounds.shape[0])._rand() * (bounds[:, 1] - bounds[:, 0]) + bounds[:, 0]
        res = minimize(lambda x: -acq_fun(x),
                      locs,
                      bounds = bounds,
                      method = "L-BFGS-B")
        
        if not res.success:
            continue

        if max_acq is None or -res.fun[0] >= max_acq:
            x_max = res.x
            max_acq = -res.fun[0]
            
    return torch.clip(x_max, bounds[:, 0], bounds[:, 1])

def bayes_opt(bounds, sur_model, evaluate, acq_fun = UCB):
    sur_model.fit()
    for pnt in range(num_start_pnt):
        next_pnt = acq_max(bounds, sur_model, acq_fun = acq_fun)
        sur_model.fit(next_pnt)
    

In [ ]:
def test_trivial():
    selector = TrivialSelector(n_kernels)
    # main model
    model = StructuralSparseGP(gps, selector)
    likelihood = GaussianLikelihood()
    elbo = VariationalELBO(likelihood, model, num_data=100)

In [8]:
model = svm.SVR(kernel='rbf')

In [10]:
model.fit(boston['data'], boston['target'])

SVR()

In [ ]:
model.predict()